<a href="https://colab.research.google.com/github/tukamilano/combinatory_logic/blob/main/combinator_logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

簡約された式のみをgenerateできるようにしといたほうが効率がいいかも？

In [ ]:
def calculator_reduce(term, reduction_order): #reduction_order is "left", "right", or "all"
  reduced = True
  if (reduction_order == "left") or (reduction_order == "all"):
    scan = range(len(term))
  else:
    scan = reversed(range(len(term)))

  next_term_list = []
  for i in scan:
    for special_token in special_token_list:
      depth = special_token_list[special_token][0]
      if (depth <= i) and (term[i-depth:i+1] == "A" * depth + special_token):
        pointer = i+1
        separator = [pointer]
        for _ in range(depth):
          num_placeholder = 1
          while num_placeholder != 0:
            if term[pointer] == "A":
              num_placeholder += 1
            else:
              num_placeholder -= 1
            pointer += 1
          separator.append(pointer)
        new_term = term[:i-depth]
        for repl in special_token_list[special_token][1]:
          if isinstance(repl, str):
            new_term += repl
          else:
            new_term += term[(separator[repl-1]):(separator[repl])]
        new_term += term[pointer:]
        reduced = False
        if (reduction_order == "left") or (reduction_order == "right"):
          return new_term, reduced
        else:
          next_term_list.append(new_term)

  if (reduction_order == "left") or (reduction_order == "right"):
    return term, reduced
  else:
    return next_term_list

In [ ]:
from itertools import product

def _generate_fixed_structure(n, sequence='', a_count=0, l_count=0, results=None):
    if results is None:
        results = []
    if len(sequence) == 2 * n:
        results.append(sequence + "V")
        return results
    if a_count < n:
        _generate_fixed_structure(n, sequence + 'A', a_count + 1, l_count, results)
    if l_count < a_count:
        _generate_fixed_structure(n, sequence + 'V', a_count, l_count + 1, results)
    return results

def _replace_variables(all_structures, token_list):
    new_sequences = []

    for sequence in all_structures:
        v_count = sequence.count("V")
        for replacement in product(token_list, repeat=v_count):
            new_sequence = sequence
            for var in replacement:
                new_sequence = new_sequence.replace("V", var, 1)
            new_sequences.append(new_sequence)

    return new_sequences

def variable_generate_sequences(token_list, length):
    all_structures = []
    for n in range(0,length):
        structure = _generate_fixed_structure(n)
        all_structures.extend(structure)

    all_sequences = _replace_variables(all_structures, token_list)

    print(f"Total number of sequences after variable replacement: {len(all_sequences)}")
    return all_sequences

In [ ]:
def repeat_reduce(term, reduction_order="left", max_repeat=10, max_length=100):
  i = 0
  reduction_history = [term]
  while i < max_repeat:
    next_term, reduced = calculator_reduce(term, reduction_order)
    i += 1
    if term != next_term:
      reduction_history.append(next_term)
    if reduced == True:
      return reduction_history
    if len(term) > max_length:
      return reduction_history
    term = next_term
  return reduction_history

def all_repeat_reduce(term, max_size=100, max_repeat=10):
  i = 0
  all_term_list = [term]
  while i < len(all_term_list) < max_repeat:
      next_term_list = calculator_reduce(all_term_list[i], reduction_order="all")
      if next_term_list == []:
        break
      for next_term in next_term_list:
        if next_term not in all_term_list:
          all_term_list.append(next_term)
      i += 1
  return all_term_list

def definitionally_equal(term1, term2, max_size=100):
  if set(all_repeat_reduce(term1, max_size)) & set(all_repeat_reduce(term2, max_size)) != set():
    return True
  else:
    return False

In [ ]:
def construct_nand(nand, term1):
  term2 = "AA" + nand + term1 + term1
  if definitionally_equal(term1, term2):
    return "failed"
  tf = definitionally_equal("AA" + nand + term1 + term2, term1)
  ft = definitionally_equal("AA" + nand + term2 + term1, term1)
  ff = definitionally_equal("AA" + nand + term2 + term2, term1)
  if tf and ft and ff:
    return (nand, term1, term2)
  else:
    return "failed"

In [ ]:
#de brujin index
#special_token_list = {"S": [3, ["A", "A", 1, 3, "A", 2, 3]], "K": [2, [1]], "I": [1, [1]]}
special_token_list = {"F": [3, ["A", "A", 3, 2, 1]], "R": [3, ["A", "A", 2, 3, 1]]}
variable_token_list = []
token_list = list(special_token_list.keys()) + variable_token_list

In [ ]:
candidate_list = variable_generate_sequences(token_list, 8)

for candidate in candidate_list:
  if definitionally_equal("AAA" + candidate + "XYZ", "AAZXY"):
    print(candidate)
    break

Total number of sequences after variable replacement: 129958
AARFR
